# 用于新闻问单的 agent v0.2

## 通用设置

In [8]:
%%time

INDEX_PATH="news-index"

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.58 µs


## 初始化准备

In [10]:
%%time
!rm -rf news-index

CPU times: user 5.18 ms, sys: 7.66 ms, total: 12.8 ms
Wall time: 122 ms


## 加载模型和嵌入

In [1]:
%%time

# 加载llm和embeddings
%run ../utils2.py

from llama_index.core import Settings

# Settings.llm=get_llm("gpt-3.5-turbo")
Settings.llm=get_llm()
Settings.embed_model=get_embedding()

CPU times: user 3.38 s, sys: 411 ms, total: 3.79 s
Wall time: 3.41 s


## 通用函数

### get_news_data

In [2]:
%%time

import requests
from gne import GeneralNewsExtractor

def get_news_data(url):
    response = requests.get(url)
    html = response.text

    extractor = GeneralNewsExtractor()
    data = extractor.extract(html, noise_node_list=[
                               '//div[@class="comment-list"]'])
    data['url']=url
    return data
    

CPU times: user 11.8 ms, sys: 0 ns, total: 11.8 ms
Wall time: 11.5 ms


### load_index

In [3]:
%%time

import os
from llama_index.core import(
    StorageContext, 
    load_index_from_storage,
    get_response_synthesizer,
    DocumentSummaryIndex,
    Document
)
from llama_index.core.node_parser import SentenceSplitter

def load_index(folder_path):
    # 考虑不存在的情况，创建空的索引
    if not os.path.exists(folder_path):
        splitter = SentenceSplitter(chunk_size=1024)
        response_synthesizer = get_response_synthesizer(
            response_mode="tree_summarize", use_async=True,
        )
        
        doc_summary_index = DocumentSummaryIndex.from_documents(
            [],
            transformations=[splitter],
            response_synthesizer=response_synthesizer,
            show_progress=True,
        )
        
        doc_summary_index.storage_context.persist(folder_path)
        
    storage_context = StorageContext.from_defaults(persist_dir=folder_path)
    return load_index_from_storage(storage_context)
    

CPU times: user 13 µs, sys: 2 µs, total: 15 µs
Wall time: 16.5 µs


### data2doc

In [4]:
%%time

def _data2doc(news_data):
    document=Document(text=news_data['content'], 
                  metadata={"title": news_data['title'],
                            'publish_time': news_data['publish_time'],
                            'author': news_data['author'],
                            'url': news_data['url'],
                            'images': news_data['images'],
                           })
    document.doc_id = document.metadata["title"]
    return document

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.34 µs


### get_news_summary_tool

In [5]:
%%time

from llama_index.core.tools import FunctionTool

def get_news_summary(url: str) -> str:
    """根据url获取新闻内容, 提示词中应提供url"""
    
    news_data=get_news_data(url)
    document=_data2doc(news_data)
    doc_summary_index=load_index(INDEX_PATH)

    if document.doc_id not in doc_summary_index.ref_doc_info:
        doc_summary_index.insert(document)
        doc_summary_index.refresh([document])
        doc_summary_index.storage_context.persist(INDEX_PATH)

    summary=doc_summary_index.get_document_summary(document.doc_id)

    prompt=f"""
    请根据{summary}, 用中文做有条理的表述。
    """
        
    return prompt

get_news_summary_tool = FunctionTool.from_defaults(
            fn=get_news_summary)

CPU times: user 371 µs, sys: 0 ns, total: 371 µs
Wall time: 375 µs


### get_query_engine_tool

In [6]:
%%time

from llama_index.core.tools import QueryEngineTool
from llama_index.core.tools.types import ToolMetadata

def get_query_engine():
    """根据提示词查询信息，提示词不应存在url"""

    # content='查询到的信息: 1. xxx, 2. xxx ...'
    # return f"根据{query}查询到信息\n\n{content}"
    doc_summary_index=load_index(INDEX_PATH)

    return doc_summary_index.as_query_engine(
        response_mode="tree_summarize", 
        streaming=True,
        # similarity_top_k=5
    )

def get_query_engine_tool():
    return QueryEngineTool.from_defaults(
        query_engine=get_query_engine(),
        name="query",
        description="根据提示词查询信息，提示词不应存在url。",
        # return_direct=True,
    )


CPU times: user 8 µs, sys: 1 µs, total: 9 µs
Wall time: 10.7 µs


## 使用

### 创建agent

In [9]:
%%time

from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools([
    get_news_summary_tool,
    get_query_engine_tool()], verbose=True)

CPU times: user 316 ms, sys: 20 ms, total: 336 ms
Wall time: 335 ms


### 发送新闻链接得到摘要信息

In [11]:
%%time

response = agent.stream_chat("https://www.guancha.cn/internation/2024_06_19_738488.shtml",chat_history=[])
response.print_response_stream()

Thought: The user has provided a URL to a news article. I need to use the get_news_summary tool to extract the summary of this article.
Action: get_news_summary
Action Input: {'url': 'https://www.guancha.cn/internation/2024_06_19_738488.shtml'}


Parsing nodes: 0it [00:00, ?it/s]

Summarizing documents: 0it [00:00, ?it/s]

Generating embeddings: 0it [00:00, ?it/s]

current doc id: 波音CEO承认公司报复“吹哨人”
Observation: 
    请根据The provided texts are about a hearing held by the US Senate Permanent Subcommittee on Investigations concerning issues at Boeing Company, particularly focusing on its safety culture and management practices. The key points include:

1. **Reputation Crisis**: The article highlights the ongoing reputation crisis faced by Boeing due to frequent accidents involving their aircraft and allegations of retaliation against whistleblowers who exposed manufacturing issues.

2. **Executive Compensation**: There is discussion about CEO Dave Calhoun's compensation, which has increased from $22.6 million in 2022 to $32.8 million in 2023, leading to criticism from Republican Senator Josh Hawley regarding the disparity between high executive pay and company issues.

3. **Whistleblower Retaliation**: The text mentions that Boeing has been accused of retaliating against whistleblowers who have come forward with information about manufacturing irregul

In [43]:
%%time

doc_summary_index=load_index(INDEX_PATH)
doc_summary_index.ref_doc_info

CPU times: user 13.9 ms, sys: 0 ns, total: 13.9 ms
Wall time: 13.5 ms


{'波音CEO承认公司报复“吹哨人”': RefDocInfo(node_ids=['f9772a5c-c610-4351-9a3c-7ef9fd4a998b', '8e67f7a8-9e9f-44e2-8f04-adaa5c35cda2', 'd62c4145-62f7-4e73-9255-6605155400c9', 'a997cdd6-3a05-4f8d-b947-ba1cb81cb56c', '68bae5b6-6e1e-432c-a4b6-150a97f2a5b5', 'f9772a5c-c610-4351-9a3c-7ef9fd4a998b', '8e67f7a8-9e9f-44e2-8f04-adaa5c35cda2', 'd62c4145-62f7-4e73-9255-6605155400c9', 'a997cdd6-3a05-4f8d-b947-ba1cb81cb56c', '68bae5b6-6e1e-432c-a4b6-150a97f2a5b5', '61d5ad1a-6266-4c3a-939b-2c7748c7f67b', 'f9772a5c-c610-4351-9a3c-7ef9fd4a998b', '8e67f7a8-9e9f-44e2-8f04-adaa5c35cda2', 'd62c4145-62f7-4e73-9255-6605155400c9', 'a997cdd6-3a05-4f8d-b947-ba1cb81cb56c', '68bae5b6-6e1e-432c-a4b6-150a97f2a5b5', '61d5ad1a-6266-4c3a-939b-2c7748c7f67b', '316df1d0-685d-40f0-8459-52eb04b393bd', 'db5f6d14-0913-4817-90be-0102eb2c12f3'], metadata={'title': '波音CEO承认公司报复“吹哨人”', 'publish_time': '2024-06-19 11:35:32', 'author': '王恺雯', 'url': 'https://www.guancha.cn/internation/2024_06_19_738488.shtml', 'images': ['https://i.guancha.cn/

### 查询索引中新闻的信息

In [12]:
%%time

response = agent.stream_chat("针对波音公司安全问题，具体指控有哪些？",chat_history=[])
response.print_response_stream()
# for token in response.response_gen:
#     print(token, end="")

Thought: 我需要查询关于波音公司的安全问题的具体指控。
Action: query
Action Input: {'input': '波音公司安全问题指控'}
Observation: 波音公司近期面临了多方面的安全问题指控和舆论压力。这些问题包括：

1. **薪酬与管理层变动**：波音首席执行官戴夫·卡尔霍恩（Dave Calhoun）的高薪引起了关注，尤其是在公司安全声誉受损的情况下。卡尔霍恩在2023年的薪酬上涨至3280万美元，较前一年增长了45%。

2. **“吹哨人”事件**：多名内部员工（即“吹哨人”）揭露了波音公司在制造环节存在的问题，并遭受了来自公司高层的威胁和报复。这些指控包括使用不符合规格的零件以及向监管机构隐瞒相关信息。一些“吹哨人”的死亡事件加剧了外界对波音公司的质疑。

3. **安全文化**：波音被指责存在“破碎的安全文化”，这可能与一系列安全事故有关，尤其是2018年和2019年间两架737 MAX 8型飞机接连发生坠机事故，造成346人死亡。卡尔霍恩在听证会上承认了公司对“吹哨人”的报复行为，并表示已对涉事主管进行了处罚或解雇。

4. **法律责任与诉讼**：遇难者家属和美国参议院常设调查小组委员会主席布卢门撒尔（Richard Blumenthal）呼吁追究波音公司的法律责任，要求其承担事故责任并接受透明度和问责制。美国司法部正在考虑对波音公司提起诉讼。

这些指控和事件表明了波音公司在安全管理和企业文化方面存在严重问题，并引发了公众、监管机构以及法律部门的广泛关注。
 波音公司近期面临的安全问题主要包括：

1. **薪酬与管理层变动**：波音首席执行官戴夫·卡尔霍恩在2023年的高薪（3280万美元）引起了公众的广泛关注，尤其是在公司安全声誉受损的情况下。他的薪酬较前一年增长了45%。

2. **“吹哨人”事件**：多名内部员工揭露了波音公司在制造环节存在的问题，并遭受了来自公司高层的威胁和报复。这些指控包括使用不符合规格的零件以及向监管机构隐瞒相关信息。一些“吹哨人”的死亡事件加剧了外界对波音公司的质疑。

3. **安全文化**：波音被指责存在“破碎的安全文化”，这可能与一系列安全事故有关，尤其是2018年和2019年间两架737 MAX 8型飞机接连发生坠机事故，造成346人死亡。卡尔霍恩在听证会上承认了

### 查询索引中不存在话题

In [13]:
%%time

response = agent.stream_chat("美国为啥要考虑恢复所有B-52核打击能力？",chat_history=[])
response.print_response_stream()

Thought: 我需要查询关于美国考虑恢复所有B-52核打击能力的原因的信息。
Action: query
Action Input: {'input': '美国为什么考虑恢复所有B-52核打击能力'}
Observation: 根据提供的信息，没有提到美国考虑恢复所有B-52核打击能力的相关内容。所提供的信息主要关注的是波音公司（Boeing）的CEO戴夫·卡尔霍恩（Dave Calhoun）在面对美国参议院常设调查小组委员会听证会时承认了公司对“吹哨人”进行了报复，以及波音公司在安全文化、管理层调整和薪酬问题上的讨论。此外，信息还提到了波音737 MAX 8型飞机的坠机事故、遇难者家属的诉求、波音公司的责任以及美国司法部可能对波音提起诉讼的可能性。

关于B-52核打击能力的问题与提供的信息内容不符，因此无法基于这些信息回答有关恢复所有B-52核打击能力的原因。要了解美国考虑恢复所有B-52核打击能力的具体原因，需要查阅其他相关资料或提供更具体的情境背景。
 关于美国考虑恢复所有B-52核打击能力的原因，我目前无法提供详细的信息。这可能与国家安全政策、军事战略调整或武器现代化计划有关。要获得更准确和具体的信息，请查阅最新的官方声明、国防预算报告或相关国际新闻报道。CPU times: user 1.25 s, sys: 255 ms, total: 1.51 s
Wall time: 9.8 s


In [14]:
%%time

response = agent.stream_chat("到2035年国家能源局将新建多少核电机组？",chat_history=[])
response.print_response_stream()

Thought: 我需要查询关于2035年中国国家能源局新建核电站机组的信息。
Action: query
Action Input: {'input': '到2035年国家能源局将新建多少核电机组'}
Observation: 无法根据提供的信息回答这个问题，因为这些信息主要关注的是埃隆·马斯克对未来智能手机的展望，并没有提及到2035年中国国家能源局计划新建多少核电站机组。需要有关中国国家能源局和核电站建设的具体数据才能提供准确的答案。
 根据目前的信息，我无法提供关于到2035年国家能源局将新建多少核电机组的具体数据。需要有关中国国家能源局和核电站建设的具体信息才能提供准确的答案。CPU times: user 882 ms, sys: 196 ms, total: 1.08 s
Wall time: 5.75 s


In [76]:
%%time

doc_summary_index=load_index(INDEX_PATH)
doc_summary_index.ref_doc_info

CPU times: user 19.3 ms, sys: 133 µs, total: 19.5 ms
Wall time: 19 ms


{'波音CEO承认公司报复“吹哨人”': RefDocInfo(node_ids=['f9772a5c-c610-4351-9a3c-7ef9fd4a998b', '8e67f7a8-9e9f-44e2-8f04-adaa5c35cda2', 'd62c4145-62f7-4e73-9255-6605155400c9', 'a997cdd6-3a05-4f8d-b947-ba1cb81cb56c', '68bae5b6-6e1e-432c-a4b6-150a97f2a5b5', 'f9772a5c-c610-4351-9a3c-7ef9fd4a998b', '8e67f7a8-9e9f-44e2-8f04-adaa5c35cda2', 'd62c4145-62f7-4e73-9255-6605155400c9', 'a997cdd6-3a05-4f8d-b947-ba1cb81cb56c', '68bae5b6-6e1e-432c-a4b6-150a97f2a5b5', '61d5ad1a-6266-4c3a-939b-2c7748c7f67b', 'f9772a5c-c610-4351-9a3c-7ef9fd4a998b', '8e67f7a8-9e9f-44e2-8f04-adaa5c35cda2', 'd62c4145-62f7-4e73-9255-6605155400c9', 'a997cdd6-3a05-4f8d-b947-ba1cb81cb56c', '68bae5b6-6e1e-432c-a4b6-150a97f2a5b5', '61d5ad1a-6266-4c3a-939b-2c7748c7f67b', '316df1d0-685d-40f0-8459-52eb04b393bd', 'db5f6d14-0913-4817-90be-0102eb2c12f3'], metadata={'title': '波音CEO承认公司报复“吹哨人”', 'publish_time': '2024-06-19 11:35:32', 'author': '王恺雯', 'url': 'https://www.guancha.cn/internation/2024_06_19_738488.shtml', 'images': ['https://i.guancha.cn/